1. Убираем у задачи условие целочисленности
2. Решаем задачу симплекс методом
3. Находим решение - если оно не целочисленное - проверяем лучше ли оно текущего решения
4. если да - делим вещественную прямую на отрезки с окружающими его целочисленными значениями началом и концом отрезков
5. записываем новые задачи с новыми условиями в стек

In [92]:
import scipy.optimize as opt
import math
import numpy as np

from IPython.display import display, Math, Latex

In [96]:
def numpy_to_pmatrix(a):
    lines = str(a).replace('[', '').replace(']', '').splitlines()
    rv = [r'\begin{pmatrix}']
    rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
    rv +=  [r'\end{pmatrix}']
    return '\n'.join(rv)

In [97]:
def is_float_vector(x):
    for idx, x_i in enumerate(x):
        if int(x_i) != float(x_i):
            return x_i, idx
    else:
        return None, None

In [156]:

def solve(A, c, b, lower_bound, upper_bound, verbose=False):
    
    current_x = None
    current_fun = float('-inf')
    stack = [{'A': A, 'c': c, 'b': b, 'lower': lower_bound, 'upper': upper_bound}]
    while stack:
        ILP = stack.pop()
        
        resolved = opt.linprog(c=-ILP['c'], 
                               A_ub=ILP['A'], 
                               b_ub=ILP['b'], 
                               bounds=np.concatenate((ILP['lower'].reshape(-1, 1), 
                                                      ILP['upper'].reshape(-1, 1)), 
                                                     axis=1), 
                               method='simplex')
        
        if resolved.success == False:
            display(Math(r'\text{Данная задача ILP: }\\A = '  + numpy_to_pmatrix(ILP['A'])))
            display(Math(r'b = '  + numpy_to_pmatrix(ILP['b'])))
            display(Math(r'c = '  + numpy_to_pmatrix(ILP['c'])))
            display(Math(r'lower bound = '  + numpy_to_pmatrix(ILP['lower'])))
            display(Math(r'upper bound = '  + numpy_to_pmatrix(ILP['upper'])))
            display(Math(r'\\\text{потому что: }\\'))
            print(resolved.message)
            continue
            
        display(Math(r'\text{О, мы нашли оптимальный план задачи: }'  + numpy_to_pmatrix(resolved.x)))
        display(Math(r'\text{Текущий максимум: }' + str(current_fun) + r'\\\text{Максимум этой задачи: }' + str(-resolved.fun)))
        number_to_round, idx = is_float_vector(resolved.x)
        if number_to_round and current_fun < -resolved.fun:
            low_copy = ILP['lower'].copy()
            up_copy = ILP['upper'].copy()
            low_copy[idx] = math.ceil(number_to_round) 
            up_copy[idx] = math.floor(number_to_round)
            
            stack.append({'A': A, 'c': c, 'b': b, 'lower': low_copy, 'upper': ILP['upper']})
            stack.append({'A': A, 'c': c, 'b': b, 'lower': ILP['lower'], 'upper': up_copy})
            
        else:
            if current_fun < -resolved.fun:
                display(Math(r'\text{О, целочисленное решение, ура! }\\x_{max}='  + str(-resolved.fun) +
                             r'\\\text{Лучший план}' + numpy_to_pmatrix(resolved.x)))
                current_fun = -resolved.fun
                current_x = resolved.x
    display(Math(r'\text{Стек задач пуст}'))
    return current_x, current_fun
            
        
        

In [157]:
A = np.array(
    [
        [4, 3],
        [-4, 3]
    ], dtype="float64",)
b = np.array([22, 2], dtype="float64")
c = np.array([-5, 4], dtype="float64")
lower_bound = np.array([1, 0], dtype="float64")
upper_bound = np.array([4, 5], dtype="float64")

A_prime = np.concatenate((A, np.eye(A.shape[0])), axis=1)
c_prime = np.concatenate((c, np.zeros(A.shape[0])), axis=0)

In [158]:
current_x, current_fun = solve(A, c, b, lower_bound, upper_bound)
display(Math(r'\text{Текущий максимум: }' + str(current_fun) + r'\\\text{Лучший план: }' + numpy_to_pmatrix(current_x)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

Phase 1 of the simplex method failed to find a feasible solution. The pseudo-objective function evaluates to 2.0e+00 which exceeds the required tolerance of 1e-09 for a solution to be considered 'close enough' to zero to be a basic solution. Consider increasing the tolerance to be greater than 2.0e+00. If this tolerance is unacceptably  large the problem may be infeasible.


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>